In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2
from collections import Counter
import random
from utils import create_dataset
from utils import load_model, save_checkpoint
from utils import AveragePrecision
import random

## PreProcess

In [34]:
food_data = pd.read_csv("data/food.csv")
rating_data = pd.read_csv("data/rating_train.csv")
user_data = pd.read_csv("data/user.csv")
food_data.replace('-', np.nan, inplace=True)
rating_data[['year', 'month', 'day']] = rating_data['date'].str.split('-', expand=True)
rating_data['date'] = rating_data['date'].str.replace('-','')
user_ids = user_data['userid'].sort_values(axis=0).reset_index(drop=True)

food_data.drop(['annotated_food_name'], axis=1, inplace=True)
for h in list(food_data):
    if h.endswith('cat'):
        food_data[h][food_data[h].notnull()] = 1
food_data.fillna(0, inplace=True)

/home/victai/.local/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [35]:
data_cnt = {}
for i in user_ids:
    data_cnt[i] = len(rating_data[rating_data['userid'] == i])

#plt.hist(list(data_cnt.values()), bins=20)
#list(data_cnt.values())

In [36]:
user_food_score = [{} for i in range(max(user_ids)+1)]
for u, f in zip(rating_data['userid'], rating_data['foodid']):
    if user_food_score[u].get(f) != None:
        user_food_score[u][f] += 1
    else:
        user_food_score[u][f] = 1
    
for i in user_ids:
    all_val = list(user_food_score[i].values())
    for k, v in user_food_score[i].items():
        user_food_score[i][k] = v / max(all_val)

In [152]:
valid_percentage = 0.5
user_id_train, user_id_valid, food_id_train, food_id_valid, Y_train, Y_valid = create_dataset(user_food_score, rating_data, valid_percentage)

order = np.arange(len(user_id_train))
random.seed(0)
random.shuffle(order)

In [38]:
not_eaten = {}
for u in user_ids:
    t_start_idx = np.where(user_id_train == u)[0][0]
    t_end_idx = np.where(user_id_train == u)[0][-1]
    v_start_idx = np.where(user_id_valid == u)[0][0]
    v_end_idx = np.where(user_id_valid == u)[0][-1]
    #tmp = []
    #for i in range(v_start_idx, v_end_idx+1):
        #if (food_id_valid[i] not in food_id_train[t_start_idx: t_end_idx+1]) and (food_id_valid[i] not in tmp):
        #    tmp.append(food_id_valid[i])
    #print(u)
    #print(t_end_idx)
    not_eaten[u] = np.setdiff1d(food_id_valid[v_start_idx: v_end_idx+1], food_id_train[t_start_idx: t_end_idx+1])

## Model

In [97]:
from keras.models import Sequential, Model, load_model
from keras.layers import Embedding, Input, merge, Flatten, Dropout
from keras.layers import Add, Reshape
from keras.optimizers import SGD
from keras.activations import relu, sigmoid

In [175]:
food_id_train

array([  0,   0,   1, ..., 402, 669,  39])

In [173]:
dim=10
idx = np.arange(len(user_id_train))
#np.random.seed(5)
#np.random.shuffle(idx)
#user_id_train = user_id_train[idx]
#food_id_train = food_id_train[idx]
Y_train = Y_train[idx]
user_in = Input(shape=(1,))
user_vec = Flatten()(Embedding(len(user_ids), dim)(user_in))
user_vec = Dropout(0.333)(user_vec)
movie_in = Input(shape=(1,))
movie_vec = Flatten()(Embedding(len(food_data), dim)(movie_in))
movie_vec = Dropout(0.3)(movie_vec)
dot = merge([user_vec, movie_vec], mode='dot')
u_bias = Reshape((1,))(Embedding(len(user_ids),1)(user_in))
m_bias = Reshape((1,))(Embedding(len(food_data),1)(movie_in))
add_bias = Add()([dot, u_bias, m_bias])
model = Model([user_in, movie_in], add_bias)
model.compile(optimizer='adam', loss='mse')
history = model.fit([user_id_train, food_id_train], Y_train, epochs=100, batch_size=512, shuffle=True, validation_data=([user_id_valid, food_id_valid], Y_valid))

/home/victai/.local/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


Train on 1340203 samples, validate on 1341291 samples
Epoch 1/100
1340203/1340203 [==============================] - 9s - loss: nan - val_loss: nan
Epoch 2/100
1080320/1340203 [=======================>......] - ETA: 1s - loss: nan

KeyboardInterrupt: 

## Inference

In [101]:
def infer(): 
    F = np.arange(len(food_data), dtype=int)
    F = [i for i in range(len(food_data))]
    all_ans = []
    all_score = []
    for u in user_ids[:5]:
        ans = []
        U = [u for i in range(len(food_data))]
        res = model.predict([np.array(U), np.array(F)])
        res = res.flatten()
        all_res.append(res)
        D = {k: v for (k, v) in zip(F, res)}
        sorted_D = sorted(D.items(), key=lambda d: d[1], reverse=True)
        for x in sorted_D:
            if len(ans) >= 20: break
            if x[0] in user_food_score[u].keys():
                continue
            ans.append(x[0])
        all_score.append(AveragePrecision(not_eaten[u], ans, len(ans)))
        all_ans.append(ans) 
    return all_score, all_ans

In [156]:
all_res = []
S, A = infer()
print(np.array(S).sum())
print(A[0])

0.0
[176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195]


In [162]:
i = 174
print(all_res[0][i])
print(user_food_score[6][i])

nan
0.011235955056179775


In [76]:
data

0.39325842696629215

## Generate Output

In [46]:
with open("pred.csv", 'w') as f:
    f.write("userid,foodid\n")
    for i in range(len(user_ids)):
        f.write("{},".format(user_ids[i]))
        for j in all_ans[i]:
            f.write("{} ".format(j))
        f.write('\n')

NameError: name 'all_ans' is not defined